# Cleaning and EDA



In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from scipy import stats

In [3]:
realEstate = pd.read_csv('Ames Real Estate Data.csv')
housePrice = pd.read_csv('Ames_HousePrice.csv')
table = pd.read_excel('Tbl_DataExportSpec.xls')

C:\Users\hzeig\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (36,40,81,87) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
realEstate.shape

(22213, 91)

In [5]:
housePrice.shape

(2580, 82)

In [6]:
table.shape   # table contains information for realEstate columns

(100, 7)

In [7]:
# cleaning housePrice (not sure if we need realEstate yet)

In [8]:
# dictionary of housePrice column meanings for quick reference
with open('housePrice_features') as hp_feat:
    hp_dict = {}
    for line in hp_feat.readlines():
        feature, description = [*line.split(':')]
        hp_dict[feature] = description

In [21]:
hp_dict['LotFrontage']

' Linear feet of street connected to property\n'

In [18]:
# imputing missing values
# columns with NaN
many_na = housePrice.loc[:, housePrice.isna().sum() > 0 ]
# many_na = housePrice.loc[:, housePrice.isna().sum() > (len(housePrice)/30)]
nas = many_na.isna().sum().sort_values(ascending = False)

In [ ]:
# investigate / impute column NaNs

In [65]:
hp_dict['PoolQC']             
housePrice.PoolQC.unique()
housePrice.PoolQC.value_counts()
housePrice.groupby('PoolQC')['SalePrice'].agg(['mean', 'std', 'count'])

,mean,std,count
PoolQC,,,
Ex,465000,212132.034356,2
Fa,215500,48790.367902,2
Gd,215500,62932.503526,2
TA,170500,51529.117982,3


In [58]:
hp_dict['MiscFeature']                                # drop
# housePrice.MiscFeature.unique()
# housePrice.MiscFeature.value_counts()
housePrice.groupby('MiscFeature')['SalePrice'].agg(['mean', 'std', 'count'])

,mean,std,count
MiscFeature,,,
Gar2,147500.000000,36335.244598,5
Othr,124666.666667,65896.383310,3
Shed,156386.363636,42726.987304,88
TenC,250000.000000,NaN,1


In [59]:
hp_dict['Alley']
# housePrice.Alley.unique()
housePrice.Alley.value_counts()
housePrice.groupby('Alley')['SalePrice'].agg(['count','mean','std'])

,count,mean,std
Alley,,,
Grvl,105,126345.990476,36101.160174
Pave,63,178559.698413,45650.146340


In [63]:
hp_dict['Fence']  
# Good Privacy - GdPr
# Good Wood - GdWo
# Minimum Privacy - MnPr
# Minimum Wood - MnWw
housePrice.groupby('Fence')['SalePrice'].agg(['count','mean','std'])

,count,mean,std
Fence,,,
GdPrv,111,176726.576577,62331.695505
GdWo,103,144750.223301,47246.448542
MnPrv,301,146485.810631,44748.238487
MnWw,10,131965.000000,22333.856337


In [61]:
hp_dict['FireplaceQu']
housePrice.groupby('FireplaceQu')['SalePrice'].agg(['count','mean','std'])

,count,mean,std
FireplaceQu,,,
Ex,35,340444.542857,112432.168836
Fa,70,168482.142857,36079.397184
Gd,619,223019.756058,91182.521787
Po,43,142528.674419,34116.467478
TA,572,204473.805944,63380.868389


In [26]:
hp_dict['MiscFeature']
housePrice.MiscFeature.unique()

array([nan, 'Shed', 'Othr', 'Gar2', 'TenC'], dtype=object)

In [19]:
nas

PoolQC          2571
MiscFeature     2483
Alley           2412
Fence           2055
FireplaceQu     1241
LotFrontage      462
GarageCond       129
GarageQual       129
GarageFinish     129
GarageYrBlt      129
GarageType       127
BsmtExposure      71
BsmtFinType2      70
BsmtFinType1      69
BsmtQual          69
BsmtCond          69
MasVnrArea        14
MasVnrType        14
BsmtHalfBath       2
BsmtFullBath       2
TotalBsmtSF        1
BsmtUnfSF          1
GarageCars         1
GarageArea         1
BsmtFinSF2         1
BsmtFinSF1         1
Electrical         1
dtype: int64

In [77]:
# LotFrontage

In [26]:
# finding columns with few unique values

two_val_columns = housePrice.loc[:, housePrice.apply(lambda x: x.nunique()) <= 2]
two_val_columns.value_counts()

Street  Alley  Utilities  CentralAir
Pave    Grvl   AllPub     Y             82
        Pave   AllPub     Y             56
        Grvl   AllPub     N             23
        Pave   AllPub     N              7
dtype: int64

In [27]:
(realEstate.isnull().sum() > 20000).sum()
# realEstate = realEstate.dropna()
# realEstate[realEstate.isnull().all()==True]

14